In [12]:
import os
import csv
import re

# PGBench Excel {'hypervisor':'', 'host':'', 'guest':'', 'ram':'', 'tps':''}

# SysBench Excel {'hypervisor':'', 'host':'', 'guest':'', 'ram':'', 'CPU Total Time':'','Mem Throughput':''}
#see https://wiki.gentoo.org/wiki/Sysbench

# iPerf {'hypervisor':'', 'host':'', 'guest':'', 'ram':'', _________} TODO: Finish the relevant fields

# fio {'hypervisor':'', 'host':'', 'guest':'', 'ram':'', 'randread runtime (ms)':'' , 'randwrite runtime (ms)':'' , 'seqread runtime (ms)':'' , 'seqwrite runtime (ms)':''}

PGBench = [] #For each test run, make a new dict and add it to this list
Sysbench = [] #For each test run, make a new dict and add it to this list
iPerf = [] #For each test run, make a new dict and add it to this list
fio = [] #For each test run, make a new dict and add it to this list

for filename in os.listdir('results'):
    
    
    #PGBench Parser
    if filename[0] == 'p':
        
        f = open('results/' +filename, "r")
        lines = f.readlines()
        hypervisor = re.search('pgbenchResults_(.*)_.*_.*_.*.txt', filename).group(1)
        if hypervisor == 'virtualbox':
            hypervisor = 'vbox'
        host = re.search('pgbenchResults_.*_(.*)_.*_.*.txt', filename).group(1)
        guest =re.search('pgbenchResults_.*_.*_(.*)_.*.txt', filename).group(1)
        ram = re.search('pgbenchResults_.*_.*_.*_(.).*.txt', filename).group(1)
        tps = 0.0
        for line in lines:
            if 'including' in line:
                tps = float(re.search('tps = (.*) \(', line).group(1))
                PGBench.append({'hypervisor':hypervisor, 'host':host, 'guest':guest, 'ram':ram, 'tps': tps})#Create dict and add to list
                tps = 0.0#Reset values
                
    CPU = 0.0
    MemOps = 0.0
    MemThroughput = 0.0
    #Sysbench Parser
    if filename[0] == 's':
        
        f = open('results/' +filename, "r")
        lines = f.readlines()
        hypervisor = re.search('sysbenchResults_(.*)_.*_.*_.*.txt', filename).group(1)
        if hypervisor == 'virtualbox':
            hypervisor = 'vbox'
        host = re.search('sysbenchResults_.*_(.*)_.*_.*.txt', filename).group(1)
        guest =re.search('sysbenchResults_.*_.*_(.*)_.*.txt', filename).group(1)
        ram = re.search('sysbenchResults_.*_.*_.*_(.).*.txt', filename).group(1)
        for line in lines:
            if 'events per second' in line:
                CPU = float(re.search('events per second:   (.*)', line).group(1))
            if '4096.00 MiB transferred (' in line:
                MemThroughput = float(re.search('4096.00 MiB transferred \((.*) MiB/sec', line).group(1))
                Sysbench.append({'hypervisor':hypervisor, 'host':host, 'guest':guest, 'ram':ram, 'CPU Total Time': CPU, 'Mem Throughput': MemThroughput})
                CPU = 0.0
                MemThroughput = 0.0
                
    if filename[0] == 'i':
        f = open('results/' + filename, "r")
        lines = f.readlines()
        
        hypervisor = re.search('iperfResults_(.*)_.*_.*_.*.txt', filename).group(1)
        if hypervisor == 'virtualbox':
            hypervisor = 'vbox'
        host = re.search('iperfResults_.*_(.*)_.*_.*.txt', filename).group(1)
        guest =re.search('iperfResults_.*_.*_(.*)_.*.txt', filename).group(1)
        ram = re.search('iperfResults_.*_.*_.*_(.).*.txt', filename).group(1)
        
        currentlySearching = "tcp_time"
        found = False

        tcp_time = 0
        udp_time = 0
        tcp_space = 0
        udp_space = 0
        
        for line in lines:
            if currentlySearching == "tcp_time":
                if "sum_received" in line:
                    found = True
                if found and "bytes" in line:
                    tcp_time = re.search('\d*\.?\d+', line)[0]
                    found = False
                    currentlySearching = "udp_time"
            elif currentlySearching == "udp_time":
                if "\"end\":\t{" in line:
                    found = True
                if found and "bytes" in line:
                    udp_time = re.search('\d*\.?\d+', line)[0]
                    found = False
                    currentlySearching = "tcp_space" 
            elif currentlySearching == "tcp_space":
                if "sum_received" in line:
                    found = True
                if found and "seconds" in line:
                    tcp_space = re.search('\d*\.?\d+', line)[0]
                    found = False
                    currentlySearching = "udp_space"
            elif currentlySearching == "udp_space":
                if "\"end\":\t{" in line:
                    found = True
                if found and "seconds" in line:
                    udp_space = re.search('\d*\.?\d+', line)[0]
                    found = False
                    currentlySearching = "tcp_time"
                    iPerf.append({'hypervisor':hypervisor, 'host':host, 'guest':guest, 'ram':ram,
                                  "tcp_time_bytes_received": tcp_time, "udp_time_bytes_received": udp_time,
                                  "tcp_space_time_elapsed": tcp_space, "udp_space_time_elapsed": udp_space})
                
                
    #FIO Parser
    if filename[0] == 'f':
        
        f = open('results/' +filename, "r")
        lines = f.readlines()
        hypervisor = re.search('fioResults_(.*)_.*_.*_.*.txt', filename).group(1)
        if hypervisor == 'virtualbox':
            hypervisor = 'vbox'
        host = re.search('fioResults_.*_(.*)_.*_.*.txt', filename).group(1)
        guest =re.search('fioResults_.*_.*_(.*)_.*.txt', filename).group(1)
        ram = re.search('fioResults_.*_.*_.*_(.).*.txt', filename).group(1)
        
        randreadRuntime=0
        randwriteRuntime=0 
        seqreadRuntime=0
        seqwriteRuntime=0 
        testCount = 0
        for line in lines:
            if 'run=' in line:
                testCount=testCount+1
                if (testCount%4)==1:
                    randreadRuntime=float( re.search('run=(.*)-', line).group(1))
                if (testCount%4)==2:
                    randwriteRuntime=float( re.search('run=(.*)-', line).group(1))
                if (testCount%4)==3:
                    seqreadRuntime=float( re.search('run=(.*)-', line).group(1))
                if (testCount%4)==0:
                    seqwriteRuntime=float( re.search('run=(.*)-', line).group(1))
                    fio.append({'randread runtime (ms)':randreadRuntime, 'randwrite runtime (ms)':randwriteRuntime , 'seqread runtime (ms)':seqreadRuntime , 'seqwrite runtime (ms)':seqwriteRuntime})
                    randreadRuntime=0
                    randwriteRuntime=0 
                    seqreadRuntime=0
                    seqwriteRuntime=0

    
def listToCSV(toCSV, filename):
    keys = toCSV[0].keys()
    with open(filename, 'w', newline='')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(toCSV)        
        
        
        
        
listToCSV(PGBench, 'PGBench.csv')
listToCSV(Sysbench, 'Sysbench.csv')
listToCSV(fio,"FIO.csv")
listToCSV(iPerf, 'iPerf.csv')


In [3]:
os.listdir('results')

['iperfResults_vmware_mac_centos_2.txt',
 'sysbenchResults_vbox_mac_debian_2.txt',
 'sysbenchResults_vmware_mac_centos_5.txt',
 'sysbenchResults_vmware_mac_centos_4.txt',
 'iperfResults_vmware_mac_debian_5.txt',
 'sysbenchResults_vmware_mac_debian_2.txt',
 'sysbenchResults_vbox_mac_centos_5.txt',
 'fioResults_vbox_windows_ubuntu_2gb.txt',
 'fioResults_vmware_windows_centos_2gb.txt',
 'sysbenchResults_vmware_windows_debian_2.txt',
 'sysbenchResults_vbox_windows_ubuntu_4gb.txt',
 'pgbenchResults_vbox_windows_centos_2gb.txt',
 'sysbenchResults_vbox_mac_centos_4.txt',
 'iperfResults_vmware_mac_debian_4.txt',
 'iperfResults_vmware_windows_centos_5gb.txt',
 'iperfResults_vmware_mac_centos_4.txt',
 'sysbenchResults_vbox_mac_debian_4.txt',
 'sysbenchResults_vbox_mac_debian_5.txt',
 'sysbenchResults_vmware_mac_centos_2.txt',
 'pgbenchResults_vmware_windows_ubuntu_2.txt',
 'iperfResults_vmware_mac_centos_5.txt',
 'sysbenchResults_vbox_windows_ubuntu_2gb.txt',
 'fioResults_vmware_windows_debian_2

In [11]:
f = open('results/iperfResults_vmware_mac_centos_2.txt')
lines = f.readlines()
currentlySearching = "tcp_time"
found = False

dicts = []

tcp_time = 0
udp_time = 0
tcp_space = 0
udp_space = 0

for line in lines:
    if currentlySearching == "tcp_time":
        if "sum_received" in line:
            found = True
        if found and "bytes" in line:
            # re search
            print("tcp time")
            tcp_time = re.search('\d*\.?\d+', line)[0]
            found = False
            currentlySearching = "udp_time"
    elif currentlySearching == "udp_time":
        if "\"end\":\t{" in line:
            found = True
        if found and "bytes" in line:
            print("udp time")
            udp_time = re.search('\d*\.?\d+', line)[0]
            found = False
            currentlySearching = "tcp_space" 
    elif currentlySearching == "tcp_space":
        if "sum_received" in line:
            found = True
        if found and "seconds" in line:
            print("tcp space")
            tcp_space = re.search('\d*\.?\d+', line)[0]
            found = False
            currentlySearching = "udp_space"
    elif currentlySearching == "udp_space":
        if "\"end\":\t{" in line:
            found = True
        if found and "seconds" in line:
            print("udp space")
            udp_space = re.search('\d*\.?\d+', line)[0]
            found = False
            currentlySearching = "tcp_time"
            dicts.append({"tcp_time_bytes_received": tcp_time, "udp_time_bytes_received": udp_time,
                          "tcp_space_time_elapsed": tcp_space, "udp_space_time_elapsed": udp_space})
print(dicts)

tcp time
udp time
tcp space
udp space
tcp time
udp time
tcp space
udp space
tcp time
udp time
tcp space
udp space
tcp time
udp time
tcp space
udp space
tcp time
udp time
tcp space
udp space
[{'tcp_time_bytes_received': '77115750', 'udp_time_bytes_received': '1310796', 'tcp_space_time_elapsed': '10.535108', 'udp_space_time_elapsed': '10.473356008529663'}, {'tcp_time_bytes_received': '77966256', 'udp_time_bytes_received': '1310796', 'tcp_space_time_elapsed': '10.160475', 'udp_space_time_elapsed': '10.47333288192749'}, {'tcp_time_bytes_received': '68613438', 'udp_time_bytes_received': '1310796', 'tcp_space_time_elapsed': '10.837072', 'udp_space_time_elapsed': '10.473344087600708'}, {'tcp_time_bytes_received': '80486172', 'udp_time_bytes_received': '1310796', 'tcp_space_time_elapsed': '10.990001', 'udp_space_time_elapsed': '10.473406076431274'}, {'tcp_time_bytes_received': '81274848', 'udp_time_bytes_received': '1310796', 'tcp_space_time_elapsed': '8.77279', 'udp_space_time_elapsed': '10.4

In [5]:
lines

['Starting new iperf Test\n',
 'Starting tcp_time test\n',
 '{\n',
 '\t"start":\t{\n',
 '\t\t"connected":\t[{\n',
 '\t\t\t\t"socket":\t5,\n',
 '\t\t\t\t"local_host":\t"172.16.255.2",\n',
 '\t\t\t\t"local_port":\t57798,\n',
 '\t\t\t\t"remote_host":\t"152.23.250.154",\n',
 '\t\t\t\t"remote_port":\t5201\n',
 '\t\t\t}],\n',
 '\t\t"version":\t"iperf 3.5",\n',
 '\t\t"system_info":\t"Linux localhost.localdomain 4.18.0-240.el8.x86_64 #1 SMP Fri Sep 25 19:48:47 UTC 2020 x86_64",\n',
 '\t\t"timestamp":\t{\n',
 '\t\t\t"time":\t"Tue, 27 Apr 2021 22:42:48 GMT",\n',
 '\t\t\t"timesecs":\t1619563368\n',
 '\t\t},\n',
 '\t\t"connecting_to":\t{\n',
 '\t\t\t"host":\t"152.23.250.154",\n',
 '\t\t\t"port":\t5201\n',
 '\t\t},\n',
 '\t\t"cookie":\t"jbeujycj4htja6sdpuez47szfhrevc6o4ux2",\n',
 '\t\t"tcp_mss_default":\t1374,\n',
 '\t\t"sock_bufsize":\t0,\n',
 '\t\t"sndbuf_actual":\t16384,\n',
 '\t\t"rcvbuf_actual":\t87380,\n',
 '\t\t"test_start":\t{\n',
 '\t\t\t"protocol":\t"TCP",\n',
 '\t\t\t"num_streams":\t1,\n